In [ ]:
import h5py 
import cupy
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture

#no test-train split; running on all images

with h5py.File('allNSIDC_fortraining.h5', "r") as f: #448 x 304 x 7300 original daily images
    X0 = f['sic'][()] 
    
X0=np.where(X0<0.40,0,X0) #change the value here to impose different SIC thresholds  
    
nlon=X0.shape[2]
nlat=X0.shape[1]
X0=cupy.reshape(X0,[X0.shape[0],nlon*nlat]).transpose()

mask_OO=np.zeros((nlat,nlon,X0.shape[1]))

for i in range(X0.shape[1]):
    junkmask=np.zeros((nlat,nlon,10))
    
    for j in range(10):
        gmm = GaussianMixture(n_components = 3, covariance_type = 'full', tol = 1e-3, max_iter = 200, n_init = 3, init_params = 'kmeans', verbose = 0)
        gmm.fit(X0[:,np.max((i,0)):np.min((i+1,X0.shape[1]))])
        Ypred = gmm.predict(X0[:,np.max((i,0)):np.min((i+1,X0.shape[1]))])
        junk=cupy.reshape(Ypred,[nlat, nlon])
        classnum=np.median(junk[370:400,150:250])
        junkmask[:,:,j]=np.where(junk==classnum,1,0)
        del gmm, junk, Ypred, classnum
        
    junkmask=np.squeeze(np.sum(junkmask,axis=2))
    mask_OO[:,:,i]=np.where(junkmask>=5,1,0)
    
    if i%500==0: #On our system, takes 20h to run on all 7300 images; we save every 500 images  
        f1 = h5py.File("GMM_fortraining_3classes_SIC40_2days.hdf5", "w")
        dset1 = f1.create_dataset("mask_OO", mask_OO.shape, dtype=np.float32, data=mask_OO)
        f1.close()
        
f1 = h5py.File("GMM_fortraining_3classes_SIC40_2days.hdf5", "w")
dset1 = f1.create_dataset("mask_OO", mask_OO.shape, dtype=np.float32, data=mask_OO)
f1.close()

0
500
1000
1500
2000
2500
